In [ ]:
from google.colab import data_table
from google.colab import files
import io
from scipy.sparse import lil_matrix
from sklearn.neighbors import NearestNeighbors
import numpy as np
import pandas as pd

Import Dataset

In [ ]:
uploaded_files = files.upload()

Saving movies_hw.txt to movies_hw.txt


In [ ]:
data = pd.read_csv(io.StringIO(uploaded_files['movies_hw.txt'].decode()),sep='\t')
#print(data)
data_table.DataTable(data, include_index=True, num_rows_per_page=10)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
number_of_movies = data['movie'].nunique()
print(number_of_movies)

4859


In [ ]:
number_of_users = data['userId'].nunique()
print(number_of_users)

610


In [ ]:
data['rating'].mean()

3.504087692574668

Rating Matrix

In [ ]:
movie_title_to_index = {}
movie_index_to_title = {}
i = 0
for movie_title in np.unique(data['movie']):
  movie_title_to_index[movie_title] = i
  movie_index_to_title[i] = movie_title
  i += 1

In [ ]:
 print(movie_title_to_index)

{"'Salem's Lot (2004)": 0, "'Tis the Season for Love (2015)": 1, "'burbs, The (1989)": 2, "'night Mother (1986)": 3, '(500) Days of Summer (2009)': 4, '*batteries not included (1987)': 5, '...And Justice for All (1979)': 6, '00 Schneider - Jagd auf Nihil Baxter (1994)': 7, '10 (1979)': 8, '10 Cloverfield Lane (2016)': 9, '10 Things I Hate About You (1999)': 10, '10,000 BC (2008)': 11, '100 Streets (2016)': 12, '101 Dalmatians (One Hundred and One Dalmatians) (1961)': 13, '101 Reykjavik (101 Reykjav‚àö‚â†k) (2000)': 14, '10th & Wolf (2006)': 15, '10th Victim, The (La decima vittima) (1965)': 16, '11\'09"01 - September 11 (2002)': 17, '12 Angry Men (1957)': 18, '12 Chairs (1971)': 19, '12 Rounds (2009)': 20, '127 Hours (2010)': 21, '13 Ghosts (1960)': 22, '13 Hours (2016)': 23, '13 Tzameti (2005)': 24, '13th Warrior, The (1999)': 25, '1492: Conquest of Paradise (1992)': 26, '16 Blocks (2006)': 27, '1776 (1972)': 28, '187 (One Eight Seven) (1997)': 29, '1941 (1979)': 30, '1984 (Nineteen E

In [ ]:
movie_title = "'Tis the Season for Love (2015)"
movie_index = movie_title_to_index[movie_title]
print(movie_index)

1


In [ ]:
print(movie_index_to_title)

{0: "'Salem's Lot (2004)", 1: "'Tis the Season for Love (2015)", 2: "'burbs, The (1989)", 3: "'night Mother (1986)", 4: '(500) Days of Summer (2009)', 5: '*batteries not included (1987)', 6: '...And Justice for All (1979)', 7: '00 Schneider - Jagd auf Nihil Baxter (1994)', 8: '10 (1979)', 9: '10 Cloverfield Lane (2016)', 10: '10 Things I Hate About You (1999)', 11: '10,000 BC (2008)', 12: '100 Streets (2016)', 13: '101 Dalmatians (One Hundred and One Dalmatians) (1961)', 14: '101 Reykjavik (101 Reykjav‚àö‚â†k) (2000)', 15: '10th & Wolf (2006)', 16: '10th Victim, The (La decima vittima) (1965)', 17: '11\'09"01 - September 11 (2002)', 18: '12 Angry Men (1957)', 19: '12 Chairs (1971)', 20: '12 Rounds (2009)', 21: '127 Hours (2010)', 22: '13 Ghosts (1960)', 23: '13 Hours (2016)', 24: '13 Tzameti (2005)', 25: '13th Warrior, The (1999)', 26: '1492: Conquest of Paradise (1992)', 27: '16 Blocks (2006)', 28: '1776 (1972)', 29: '187 (One Eight Seven) (1997)', 30: '1941 (1979)', 31: '1984 (Ninete

In [ ]:
ratings_matrix = lil_matrix((number_of_movies,number_of_users))
for index,row in data.iterrows():
  userId = row['userId']
  movie_title = row['movie']
  movie_index = movie_title_to_index[movie_title]
  rating = row['rating']
  ratings_matrix[movie_index,userId] = rating

In [ ]:
ratings_matrix

<4859x610 sparse matrix of type '<class 'numpy.float64'>'
	with 51860 stored elements in List of Lists format>

In [ ]:
user_id = 32
movie_title = "American President, The (1995)"
movie_index = movie_title_to_index[movie_title]
ratings_matrix[movie_index,user_id]

2.0

In [ ]:
movie_title = "Kill Bill: Vol. 2 (2004)"
movie_index = movie_title_to_index[movie_title]
number_of_ratings = ratings_matrix[[movie_index]].count_nonzero()
avg_rating = ratings_matrix[[movie_index]].sum()/number_of_ratings
print(movie_title)
print('avg rating = '+str(avg_rating))
print('# ratings = '+str(number_of_ratings))

Kill Bill: Vol. 2 (2004)
avg rating = 3.868181818181818
# ratings = 110


Creating a KNN Model

In [ ]:
knn = NearestNeighbors()
knn.fit(ratings_matrix)

NearestNeighbors()

In [ ]:
movie_title =   "Zoolander 2 (2016)"
k = 11
movie_index = movie_title_to_index[movie_title]
movie_ratings = ratings_matrix[[movie_index]]
neighbor_indexes = knn.kneighbors(movie_ratings,n_neighbors=k,return_distance=False)
print(neighbor_indexes)

[[4853 2412 3691 3204 4056 1411 3190 1423 3100  327 1665]]


In [ ]:
movie_title =   "Zoolander 2 (2016)"
k = 11
movie_index = movie_title_to_index[movie_title]
movie_ratings = ratings_matrix[[movie_index]]
neighbor_indexes = knn.kneighbors(movie_ratings,n_neighbors=k,return_distance=False)
list_of_suggestions = []
for i in range(0,k):
  neighbor_title = movie_index_to_title[neighbor_indexes.item(i)]
  list_of_suggestions.append(neighbor_title)
print(list_of_suggestions)

['Zoolander 2 (2016)', 'Knights of Badassdom (2013)', 'Savages (2012)', 'Over the Top (1987)', 'Stretch (2014)', 'Everything Must Go (2010)', 'Our Brand Is Crisis (2015)', 'Exodus: Gods and Kings (2014)', 'Notorious (2009)', 'Atomic Blonde (2017)', 'Game Over, Man! (2018)']
